In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('twitter.csv')

In [2]:
df = df.iloc[:,2:]
df

,sentiment,review
0,Positive,im getting on borderlands and i will murder yo...
1,Positive,I am coming to the borders and I will kill you...
2,Positive,im getting on borderlands and i will kill you ...
3,Positive,im coming on borderlands and i will murder you...
4,Positive,im getting on borderlands 2 and i will murder ...
...,...,...
16011,Positive,I am totally and dota 2 great big stuff - <unk>
16012,Positive,I am safe totally and dota that 2 great big st...
16013,Positive,I am totally and dota 2 great big stuff - my
16014,Irrelevant,Cloud9 is my favorite organization but they re...


In [3]:
df['sentiment'] = df['sentiment'].apply(lambda x: 0 if x == 'Negative' else 1)
df

,sentiment,review
0,1,im getting on borderlands and i will murder yo...
1,1,I am coming to the borders and I will kill you...
2,1,im getting on borderlands and i will kill you ...
3,1,im coming on borderlands and i will murder you...
4,1,im getting on borderlands 2 and i will murder ...
...,...,...
16011,1,I am totally and dota 2 great big stuff - <unk>
16012,1,I am safe totally and dota that 2 great big st...
16013,1,I am totally and dota 2 great big stuff - my
16014,1,Cloud9 is my favorite organization but they re...


In [5]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    words = word_tokenize(text)
    filtered_tweet = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_tweet)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
import re
import nltk

def preprocess(text):
    text = str(text)
    text = ' '.join(text.split())
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = remove_stopwords(text)

    return text

In [8]:
nltk.download('punkt')
df['review'] = df['review'].apply(preprocess)
df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,sentiment,review
0,1,im getting borderlands murder
1,1,coming borders kill
2,1,im getting borderlands kill
3,1,im coming borderlands murder
4,1,im getting borderlands 2 murder
...,...,...
16011,1,totally dota 2 great big stuff unk
16012,1,safe totally dota 2 great big stuff casey
16013,1,totally dota 2 great big stuff
16014,1,cloud9 favorite organization really need spend...


In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'] ,
                                   random_state=104,
                                   test_size=0.1,
                                   shuffle=True)

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)

x = vectorizer.fit_transform(df['review'])

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, df['sentiment'] ,
                                   random_state=104,
                                   test_size=0.1,
                                   shuffle=True)

In [12]:
X_test

<1602x5576 sparse matrix of type '<class 'numpy.float64'>'
	with 14734 stored elements in Compressed Sparse Row format>

In [13]:
import time
from sklearn import svm
from sklearn.metrics import classification_report

classifier_linear = svm.SVC(kernel='linear')
classifier_linear.fit(X_train, y_train)
prediction_linear = classifier_linear.predict(X_test)

In [14]:
report = classification_report(y_test, prediction_linear, output_dict=True)

In [15]:
report

{'0': {'precision': 0.9391727493917275,
  'recall': 0.8177966101694916,
  'f1-score': 0.8742921857304643,
  'support': 472},
 '1': {'precision': 0.927791771620487,
  'recall': 0.9778761061946902,
  'f1-score': 0.9521757862990091,
  'support': 1130},
 'accuracy': 0.9307116104868914,
 'macro avg': {'precision': 0.9334822605061073,
  'recall': 0.897836358182091,
  'f1-score': 0.9132339860147367,
  'support': 1602},
 'weighted avg': {'precision': 0.9311449685668202,
  'recall': 0.9307116104868914,
  'f1-score': 0.9292288078543441,
  'support': 1602}}